In [56]:
#使用requests库、pyquery库下载、抽取 可可英语双语新闻 科技分类下的文章列表第1页和其中的所有文章信息（包括元数据）； 
#将上述抽取的文章数据，以json字符串格式存入文件中。 要求存入文件的json字符串是由每篇文章作为一个字典构成的列表转化而来。
import requests
import random
import time
from pyquery import PyQuery as pq

class ProxyPool():
    def __init__(self):
        # 初始化读取proxy站点配置文件

        # 初始化读取proxy池存储位置（文件、数据库、NoSQL...)

        # 定时扫描proxy可用性、删除失效代理
        pass

    def check_a_proxy(self):
        pass

class KKBaseDownloader():
    def __init__(self):
        # 初始化代理池对象
        self.proxyp = ProxyPool()
        # 初始化headers配置列表文件路径
        self.headers_cfg_pth=''
        # 初始化最小、最大暂停间隔
        self.interval_min_max = (5,30)
        pass
    
    def gen_an_ua(self):
        self.ua_list = ["Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0"]

        return random.choice(self.ua_list)

    def get_a_proxy(self):
        proxys = [

        ]
        idx = random.randint(1,len(proxys))
        return proxys[idx]

class KKNavDl(KKBaseDownloader):
    def __init__(self,init_url):
        super(KKNavDl,self).__init__()
        self.url_tgt = init_url

    def fetch_html(self):
        ua = self.gen_an_ua()
        headers = {'User-Agent':ua}
        # _proxy = self.get_a_proxy()
        # r = requests.get(self.url_tgt,proxies=_proxy)
        r = requests.get(self.url_tgt,headers=headers)
        if r.status_code==200:
            if r.encoding == 'ISO-8859-1':
                encodings = requests.utils.get_encodings_from_content(r.text)
                if encodings:
                    encoding = encodings[0]
                else:
                    encoding = r.apparent_encoding
            encode_content = r.content.decode(encoding, 'replace').encode('utf-8', 'replace')
            return encode_content
        else:
            return ''

class KKBaseExtractor():
    def __init__(self,html='<html></html>'):
        self.doc = pq(html)
        
class KKContentEnExt(KKBaseExtractor): # 英文获取   根据class
    def __init__(self,html):
        super(KKContentEnExt,self).__init__(html)
    
    def parse(self):
        lis = self.doc('#article_eng>div')
        content = ''
        for i in range(0,len(lis)):
            pq_li = pq(lis[i])
            content += str(pq_li.find('.qh_en').text())
        return content

class KKContentZgExt(KKBaseExtractor): # 中文获取   根据class
    def __init__(self,html):
        super(KKContentZgExt,self).__init__(html)
    
    def parse(self):
        lis = self.doc('#article_eng>div')
        for i in range(0,len(lis)):
            content = ''
            pq_li = pq(lis[i])
            content += str(pq_li.find('.qh_zg').text())
        return content

class KKNavExt(KKBaseExtractor):
    def __init__(self,html):
        super(KKNavExt,self).__init__(html)

    def parse(self):
        lis = self.doc('#menu-list>li')
        nav_links = []
        for i in range(len(lis)):
            pq_li = pq(lis[i])
            title = str(pq_li.find('h2 a').attr('title'))
            titlelink=pq_li.find('h2 a').attr('href')
            t=len(str(pq_li('a').text()))
            p=len(str(pq_li('p').text()))
            #掐头去尾切片得到摘要
            abstract=str(pq_li('li').text()[t:-p-1])
            date=str(pq_li('p').text()[:-13])
            person=str(pq_li('p').text()[-9:-4])
            nav_links.append({'title':title,'link':titlelink,'content':abstract,'date':date,'person':person})
        return nav_links



    
navDl = KKNavDl(init_url='http://www.kekenet.com/read/news/keji/')
html = navDl.fetch_html()
navExt = KKNavExt(html)
nav_links = navExt.parse()
for lnk in nav_links:
    print(lnk)

{'title': '大疆要研发地上跑的无人机', 'link': 'http://www.kekenet.com/read/202001/603174.shtml', 'content': '谈到航拍无人机，DJI无疑是领军品牌。但是很少有公司尝试过移动陆上相机平台。如果说这项装有稳定摄像头的小型越野车能算作某种迹象的话，这家公司可能在朝这个方向发展。', 'date': '2020-01-08', 'person': 'Kelly'}
{'title': '中国APP正在失去一群重要用户', 'link': 'http://www.kekenet.com/read/202001/603153.shtml', 'content': '去年，印度下载量最多的100个安卓App中，有多达44个安卓App是中国公司开发的。但是今年情况有所改变，本地开发商开始反击。', 'date': '2020-01-08', 'person': 'Kelly'}
{'title': 'NASA公布了2020年火星漫游车', 'link': 'http://www.kekenet.com/read/202001/603152.shtml', 'content': 'NASA的科学家周五揭晓了2020年火星漫游车。该漫游车将在今年启航前往火星，不仅会追寻远古生命的踪迹，还会为未来人类的使命铺路。', 'date': '2020-01-07', 'person': 'Kelly'}
{'title': '全球首个电动飞机成功飞行15分钟', 'link': 'http://www.kekenet.com/read/201912/602587.shtml', 'content': '世界上第一架全电动飞机成功飞行了近15分钟。', 'date': '2019-12-26', 'person': 'Kelly'}
{'title': '苹果最新电脑屏幕只能用配套的布来擦', 'link': 'http://www.kekenet.com/read/201912/602542.shtml', 'content': '如果你觉得售价7000美元的苹果Pro Display XDR显示器的故事已经荒谬至极了，那准备好听众所周知的更吸引人的故事吧：苹果公司坚称，只能使用与显示器配套的独一无二的特殊擦镜

In [55]:
#写入json
import json
with open('week11可可英语科技.txt',"w",encoding='utf-8') as f:
    json.dump(nav_links,f)
f.close()